In [1]:
import os
import gc
import sys
import argparse
import time
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import utils, transforms, datasets
from torchvision.models.detection.rpn import AnchorGenerator

from transforms import ToTensor, Resize, Compose, testTensor
from data.kitti_dataset import KITTI
from backbone.backbone_vgg import vgg16, vgg11, vgg13, vgg19
from backbone.backbone_resnet import resnet18, resnet34
from models.faster_rcnn_mod import FasterRCNN
from models.mask_rcnn_mod import MaskRCNN
from models.keypoint_rcnn_mod import KeypointRCNN
from models.retinanet_mod import RetinaNet

from ID.intrinsic_dimension import estimate, block_analysis
from scipy.spatial.distance import pdist,squareform

from tqdm import tqdm
from PIL import Image

In [2]:
backbone = vgg16(pretrained=False).features
backbone.out_channels = 512
anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),), aspect_ratios=((0.5,1.0,2.0),)) 
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'], output_size=7, sampling_ratio=2)

In [3]:
root = os.getcwd()
model_path = os.path.join(root, "trained_model", "kitti", "retinanet", "vgg16")

In [4]:
#model = FasterRCNN(backbone, num_classes=10,rpn_anchor_generator= anchor_generator, box_roi_pool=roi_pooler )
model = RetinaNet(backbone, num_classes=10, anchor_generator=anchor_generator)
model.load_state_dict(torch.load(os.path.join(model_path, 'model.pt'), map_location=torch.device('cpu')))

<All keys matched successfully>

In [5]:
model.eval()

RetinaNet(
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dila

In [6]:
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

backbone.0.weight 	 torch.Size([64, 3, 3, 3])
backbone.0.bias 	 torch.Size([64])
backbone.2.weight 	 torch.Size([64, 64, 3, 3])
backbone.2.bias 	 torch.Size([64])
backbone.5.weight 	 torch.Size([128, 64, 3, 3])
backbone.5.bias 	 torch.Size([128])
backbone.7.weight 	 torch.Size([128, 128, 3, 3])
backbone.7.bias 	 torch.Size([128])
backbone.10.weight 	 torch.Size([256, 128, 3, 3])
backbone.10.bias 	 torch.Size([256])
backbone.12.weight 	 torch.Size([256, 256, 3, 3])
backbone.12.bias 	 torch.Size([256])
backbone.14.weight 	 torch.Size([256, 256, 3, 3])
backbone.14.bias 	 torch.Size([256])
backbone.17.weight 	 torch.Size([512, 256, 3, 3])
backbone.17.bias 	 torch.Size([512])
backbone.19.weight 	 torch.Size([512, 512, 3, 3])
backbone.19.bias 	 torch.Size([512])
backbone.21.weight 	 torch.Size([512, 512, 3, 3])
backbone.21.bias 	 torch.Size([512])
backbone.24.weight 	 torch.Size([512, 512, 3, 3])
backbone.24.bias 	 torch.Size([512])
backbone.26.weight 	 torch.Size([512, 512, 3, 3])
backbone.

In [7]:
class test_data(object):
    def __init__(self, path, transforms):
        self.path = path
        self.transforms = transforms
        self.imgs = list(sorted(os.listdir(path)))
       
    
    def __getitem__(self, idx):
        print(idx)
        img_path = os.path.join(self.path, self.imgs[idx])
        
        image = Image.open(img_path).convert('RGB')
        if self.transforms is not None:
            image = self.transforms(image)
        return image
    
    def __len__(self):
        return len(self.imgs)

In [8]:
image_directory = 'D:/Dataset/KITTI/testing/horizontal_shift'

data = test_data(image_directory, transforms= testTensor())
indices = torch.randperm(len(data))
dataset = torch.utils.data.Subset(data, indices[:24])

def collate_fn(batch):
    return tuple(zip(*batch))
    

In [9]:
testdata = DataLoader(dataset, batch_size=1, shuffle=True)

In [14]:
image = next(iter(testdata))

tensor(1076)
tensor(4040)
tensor(1017)
tensor(4804)


In [15]:
out0 = image
print(out0.shape)
out1 = model.backbone[4](F.relu(model.backbone[2](F.relu(model.backbone[0](out0)))))
print(out1.shape)
out2 = model.backbone[9](F.relu(model.backbone[7](F.relu(model.backbone[5](out1)))))
print(out2.shape)
out3 = model.backbone[16](F.relu(model.backbone[14](F.relu(model.backbone[12](F.relu(model.backbone[10](out2)))))))        
print(out3.shape)
out4 = model.backbone[23](F.relu(model.backbone[21](F.relu(model.backbone[19](F.relu(model.backbone[17](out3)))))))    
print(out4.shape)
out5 = model.backbone[30](F.relu(model.backbone[28](F.relu(model.backbone[26](F.relu(model.backbone[24](out4)))))))
print(out5.shape)
out6 = F.relu(model.head.classification_head.conv[6](F.relu(model.head.classification_head.conv[4](F.relu(model.head.classification_head.conv[2](F.relu(model.head.classification_head.conv[0](out5))))))))
print(out6.shape)
out7 = model.head.classification_head.cls_logits(out6)
print(out7.shape)
out8 = F.relu(model.head.regression_head.conv[6](F.relu(model.head.regression_head.conv[4](F.relu(model.head.regression_head.conv[2](F.relu(model.head.regression_head.conv[0](out5))))))))
print(out8.shape)
out9 = model.head.regression_head.bbox_reg(out8)
print(out9.shape)

torch.Size([4, 3, 1200, 1200])
torch.Size([4, 64, 600, 600])
torch.Size([4, 128, 300, 300])
torch.Size([4, 256, 150, 150])
torch.Size([4, 512, 75, 75])
torch.Size([4, 512, 37, 37])
torch.Size([4, 512, 37, 37])
torch.Size([4, 150, 37, 37])
torch.Size([4, 512, 37, 37])
torch.Size([4, 60, 37, 37])


In [12]:
def computeID(r, nres, fraction):
    ID = []
    n = int(np.round(r.shape[0]*fraction))
    print(n)
    dist = squareform(pdist(r, 'euclidean'))
    print(dist)
    for i in range(nres):
        dist_s = dist
        perm = np.random.permutation(dist.shape[0])[0:n]
        print(perm)
        dist_s = dist_s[perm,:]
        dist_s = dist_s[:,perm]
        print(dist_s)
        ID.append(estimate(dist_s)[2])
    mean = np.mean(ID)
    error = np.std(ID)
    return mean,error

In [13]:
Out0 = out0.view(image.shape[0], -1).cpu().data
Out1 = out1.view(image.shape[0], -1).cpu().data
Out2 = out2.view(image.shape[0], -1).cpu().data
Out3 = out3.view(image.shape[0], -1).cpu().data
Out4 = out4.view(image.shape[0], -1).cpu().data
Out5 = out5.view(image.shape[0], -1).cpu().data
Out6 = out6.view(image.shape[0], -1).cpu().data
Out7 = out7.view(image.shape[0], -1).cpu().data
Out8 = out8.view(image.shape[0], -1).cpu().data
Out9 = out9.view(image.shape[0], -1).cpu().data
#Out10 = out10.view(image.shape[0], -1).cpu().data
#Out11 = out11.view(image.shape[0], -1).cpu().data

In [16]:
Out0 = torch.cat((Out0, out0.view(image.shape[0], -1).cpu().data),0)
Out1 = torch.cat((Out1, out1.view(image.shape[0], -1).cpu().data),0) 
Out2 = torch.cat((Out2, out2.view(image.shape[0], -1).cpu().data),0) 
Out3 = torch.cat((Out3, out3.view(image.shape[0], -1).cpu().data),0)                 
Out4 = torch.cat((Out4, out4.view(image.shape[0], -1).cpu().data),0) 
Out5 = torch.cat((Out5, out5.view(image.shape[0], -1).cpu().data),0) 
Out6 = torch.cat((Out6, out6.view(image.shape[0], -1).cpu().data),0) 
Out7 = torch.cat((Out7, out7.view(image.shape[0], -1).cpu().data),0)                 
Out8 = torch.cat((Out8, out8.view(image.shape[0], -1).cpu().data),0)
Out9 = torch.cat((Out9, out9.view(image.shape[0], -1).cpu().data),0) 
#Out10 = torch.cat((Out10, out10.view(image.shape[0], -1).cpu().data),0)                 
#Out11 = torch.cat((Out11, out11.view(image.shape[0], -1).cpu().data),0)

In [17]:
out = [Out0, Out1, Out2, Out3, Out4, Out5, Out6, Out7, Out8, Out9]

In [18]:
ID_all = []
for j in tqdm(range(0,12)):
    r = out[j]
    print(r)
    ID_all.append(computeID(r, 20, 0.9))
ID_all = np.array(ID_all)

  0%|          | 0/12 [00:00<?, ?it/s]

tensor([[1.0000, 1.0000, 1.0000,  ..., 0.3451, 0.3412, 0.3373],
        [0.8784, 0.8745, 0.8706,  ..., 0.0314, 0.0314, 0.0314],
        [0.4039, 0.4275, 0.4078,  ..., 1.0000, 1.0000, 1.0000],
        ...,
        [1.0000, 1.0000, 1.0000,  ..., 0.3451, 0.3412, 0.3373],
        [0.3686, 0.3451, 0.2627,  ..., 0.2353, 0.2471, 0.2510],
        [0.2863, 0.3020, 0.3373,  ..., 0.8000, 0.7961, 0.7882]])
7
[[   0.          806.8415167  1122.34390633  778.71728191  687.7001802
     0.          839.41353936  886.70098792]
 [ 806.8415167     0.         1141.2520599   912.16916656  701.88973729
   806.8415167  1059.82332949 1127.53435898]
 [1122.34390633 1141.2520599     0.         1197.84663387  973.48172084
  1122.34390633 1081.65726794  915.29330917]
 [ 778.71728191  912.16916656 1197.84663387    0.          885.66762548
   778.71728191  959.89835336 1065.16496374]
 [ 687.7001802   701.88973729  973.48172084  885.66762548    0.
   687.7001802   902.30639117  949.04501407]
 [   0.          806.841

  0%|          | 0/12 [00:01<?, ?it/s]


ValueError: x and y must have length at least 2.

In [25]:
ID_all

[(26.09362889859936, 3.552713678800501e-15),
 (11.292461444869833, 3.552713678800501e-15),
 (11.937158710242844, 0.0),
 (7.698880093808394, 1.7763568394002505e-15),
 (11.654509692911462, 0.0),
 (34.401205237992244, 0.0),
 (18.871602110210368, 7.105427357601002e-15),
 (21.29145392917633, 0.0),
 (20.512634611260175, 3.552713678800501e-15),
 (93.92338438867301, 1.4210854715202004e-14)]

1